In [2]:
import os 
from pathlib import Path
from miditok import REMI, TokenizerConfig

In [6]:
BASE_DIR = '../'

In [4]:
BEAT_RES = {(0, 1): 12, (1, 2): 4, (2, 4): 2, (4, 8): 1}

TOKENIZER_PARAMS = {
    "pitch_range": (21, 109),
    "beat_res": BEAT_RES,
    "num_velocities": 24,
    "special_tokens": ["PAD", "BOS", "EOS"],
    "use_chords": True,
    "use_rests": True,
    "use_tempos": True,
    "use_time_signatures": True,
    "use_programs": False,  # no multitrack here
    "num_tempos": 32,
    "tempo_range": (50, 200),  # (min_tempo, max_tempo)
}

In [7]:
config = TokenizerConfig(**TOKENIZER_PARAMS)
tokenizer = REMI(config)

# Trains the tokenizer with Byte Pair Encoding (BPE) to build the vocabulary, here 30k tokens
midi_paths = list(Path("../data/raw").resolve().glob("**/*.mid"))
tokenizer.train(
    vocab_size=30000,
    files_paths=midi_paths,
)
tokenizer.save(os.path.join(BASE_DIR, 'data', 'processed', 'tokenizer_config.json'))